In [1]:
import torch
from torch import nn
import torch.utils.data as Data
import torch.nn.functional as F
from torch.autograd import Variable 
import pandas as pd
from sklearn import preprocessing
import numpy as np
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv')
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x_train = torch.FloatTensor(df.iloc[:, 1:].values)/255
x_train = x_train.view(x_train.shape[0], 1, 28, 28).cuda()

In [4]:
y_train = df['label'].values
y_train = torch.LongTensor(y_train).cuda()

In [5]:
dataset = Data.TensorDataset(x_train, y_train)
dataloader = Data.DataLoader(dataset=dataset, batch_size=128, shuffle=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv2d(1, 64, kernel_size=(2,2), stride=1, padding=(1,1))
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.cnn2 = nn.Conv2d(64, 32, kernel_size=(2,2), stride=1, padding=(1,1))
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.flatten = nn.Flatten()
        self.bn =nn.BatchNorm1d(32*7*7)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(32*7*7, 32)
        self.linear2 = nn.Linear(32, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self,x):
        x = F.relu(self.cnn1(x))
        x = self.pool1(x)
        x = F.relu(self.cnn2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.bn(x)
        x = self.dropout(x)
        x = F.relu(self.linear(x))
        x = self.linear2(x)
        x = self.softmax(x)
        return x

In [7]:
cnn = CNN().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)

In [8]:
epochs = 50
for epoch in range(epochs):
    for x, y in dataloader:
        y_pred = cnn(x)
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(loss.item())

1.590635061264038
1.502122163772583
1.5225603580474854
1.596465826034546
1.5236504077911377
1.5236505270004272
1.5236505270004272
1.4611512422561646
1.4611531496047974
1.4611505270004272
1.4611505270004272
1.5861504077911377
1.5861504077911377
1.5861505270004272
1.461154818534851
1.5236018896102905
1.4611505270004272
1.5236505270004272
1.5861504077911377
1.4611505270004272
1.4611505270004272
1.5905117988586426
1.6486504077911377
1.5236505270004272
1.5236505270004272
1.5861505270004272
1.5236505270004272
1.4611505270004272
1.6486504077911377
1.4611505270004272
1.5236505270004272
1.4611505270004272
1.4611505270004272
1.5861504077911377
1.5861451625823975
1.5236504077911377
1.4611505270004272
1.4611505270004272
1.4611505270004272
1.5236505270004272
1.4611505270004272
1.5861504077911377
1.5236505270004272
1.4611505270004272
1.4716154336929321
1.5861504077911377
1.5236505270004272
1.6486504077911377
1.4611505270004272
1.5236505270004272


In [9]:
df = pd.read_csv('test.csv')
df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
test = df.values/255
test = test.reshape(test.shape[0], 1, 28, 28)
test = torch.FloatTensor(test).cuda()

In [12]:
cnn.train(False)
ans = []
with torch.no_grad():
    for data in test:
        data = data.view(1,1,28,28)
        pred = cnn(data)
        _, pred = torch.max(pred, 1)
        pred = pred.cpu().numpy()[0]
        ans.append(pred)

In [13]:
result = pd.DataFrame(columns=['ImageId'], data=np.arange(1, len(ans)+1))
result['Label'] = ans
result

,ImageId,Label
0,1,2
1,2,3
2,3,9
3,4,7
4,5,3
...,...,...
27995,27996,3
27996,27997,7
27997,27998,3
27998,27999,9


In [17]:
result.to_csv('submission.csv', index=False)